In [11]:
#export
from nbexp_personal import sendEmail

## utils

In [12]:
#export
def itemgetter(*args):
    g = operator.itemgetter(*args)
    def f(*args2):
        return dict(zip(args,  g(*args2)))
    return f

In [13]:
#export
def write_json(filename, content):
    with open(filename, 'w', encoding='UTF-8') as f:
        json.dump( content,f, ensure_ascii=False, indent=4)
def read_json(filename):
    with open(filename, 'r', encoding='UTF-8') as f:
        return json.load( f)

## read and convert 

In [49]:
#export
# 复制为cCURL（posix）
def read_code(code_path):
    with open(code_path, 'r', encoding='UTF-8') as f:
        code = f.read().split('\n')[0]
        return code
code =read_code('../bili_curl.txt')

In [56]:
# code

In [35]:
# !{code.strip()}

In [36]:
#export
import uncurl
import requests
from functools import partial
def fetch_code(code):
    c = uncurl.parse(code)
    r = eval(c)
    j = r.json()
    return j

In [37]:
#export
import operator
import json
import datetime
def get_time(timestamp):
    d = datetime.datetime.fromtimestamp(timestamp)
    d = d.isoformat()
    return d
    
def cvt_cards(j):
    cards = j['data']['cards']
    cards = list(map(operator.itemgetter('card'), cards))
    cards = list(map(json.loads, cards))
    kl = ('title', 'desc',  'pic', 'stat', 'ctime')
    cards = list(map(itemgetter(*kl), cards))
    def cvt(card):
        content_id = str(card['stat']['aid'])
        content = itemgetter(*kl[:-2])(card)
        pic = content['pic'] + '@64w_36h_1c.jpg'
        content['pic'] = pic
        d = get_time(card['ctime'])
        url = 'https://www.bilibili.com/video/av' + content_id 
        return (content_id,  {'content': content , "url":url , 'time':d} )
    cards = dict((map(cvt, cards)))
    return cards

## render

In [38]:
#export
def render_div(v):
    content = v['content']
    desc = content['desc']

    if len(desc) > 50:
        desc = desc[:20]+'...'
    
    body = f"""
        <div style="margin:10px">
            <img src='{content['pic']}'>
            <a href='{v['url']}'>{content['title']}</a>
            <span>{desc} {v['time']}</span>
        </div>
    """
    return body

In [39]:
#export
def render_html(v_list):
    divs = ''.join(map(render_div, v_list))
    html = f"""\
    <html>
      <head></head>
      <body>
          {divs}
      </body>
    </html>
    """
    return html

In [40]:
#export
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

# Create the body of the message (a plain-text and an HTML version).
def render_msg(v_list, sub_name=""):
    v_list = list(v_list)
    html = render_html(v_list)
    msg = MIMEMultipart('alternative')
    msg['Subject'] = sub_name + '订阅' + '+' + str(len(v_list))
    msg['From'] = sub_name
    msg['To'] = ''
    # Record the MIME types of both parts - text/plain and text/html.
    part2 = MIMEText(html, 'html')
    msg.attach(part2)
    return msg.as_string()

In [41]:
# msg = render_msg(cards.values())

## check oldcards

In [42]:
#export
def get_cards():
    fetch = partial(fetch_code, code)
    cards = cvt_cards(fetch())
    return cards
    
def get_main(json_path, get_cards, sub_name=""):
    
    def main():
        cards = get_cards()
        wj = partial( write_json, json_path,)
        rj = partial( read_json, json_path,)

        if not exists(json_path):
            # 发送所有
            wj({})

        old_cards = rj()

        new_cards = filter(lambda tp:tp[0] not in old_cards, cards.items())
        new_cards = map(operator.itemgetter(1), new_cards)
        new_cards = list(new_cards)
        if new_cards:
            msg = render_msg(new_cards, sub_name)
            sendEmail(msg)
            old_cards.update(cards)
            wj(old_cards)
            
    return main

In [43]:
#export
from os.path import exists
json_path = './bili.json'
main = get_main(json_path, get_cards, "bili")


In [53]:
main()

In [29]:
#export
if __name__ == '__main__': main()

In [55]:
!python notebook2script.py bilibili.ipynb

Converted bilibili.ipynb to nbexp_bilibili.py


In [46]:
rj = partial( read_json, json_path,)
cards = rj()
# new_cards = map(operator.itemgetter(1), cards.items())
# new_cards = list(new_cards )
# msg = render_msg(new_cards, 'bili')
# sendEmail(msg, )
cards

{'93211769': {'content': {'title': '【精彩回顾】李玲：中国人究竟需要什么样的医疗制度？',
   'desc': '一勺思想·李玲：关于中国医疗体系的讨论',
   'pic': 'https://i1.hdslb.com/bfs/archive/39ebde04b813e5e2c804749938495b265860526d.jpg@64w_36h_1c.jpg'},
  'url': 'https://www.bilibili.com/video/av93211769',
  'time': '2020-03-02T21:26:31'},
 '93351316': {'content': {'title': '【这就是中国】我们找到了一位在武汉方舱医院的工作者，与他连线',
   'desc': '张维为《这就是中国》第四十六集精彩片段5',
   'pic': 'https://i1.hdslb.com/bfs/archive/27481d761d9a389a78efc7dc0391c4fc033393cf.jpg@64w_36h_1c.jpg'},
  'url': 'https://www.bilibili.com/video/av93351316',
  'time': '2020-03-03T14:15:05'},
 '93198272': {'content': {'title': '【看电影了没】别忘了，你只是个天才数学家《阿基米德大战》',
   'desc': '怎么让不认输的日本人悬崖勒马？\n阿基米德大战 アルキメデスの大戦 (2019)',
   'pic': 'https://i2.hdslb.com/bfs/archive/83b70603e2ef290ebdfdc82566c88e31c63cac49.jpg@64w_36h_1c.jpg'},
  'url': 'https://www.bilibili.com/video/av93198272',
  'time': '2020-03-03T12:24:00'},
 '93182378': {'content': {'title': '华农兄弟：捡个石头，把刀磨利，砍点兄弟家的棕花来吃',
   'desc': '华农兄弟：捡个石头，把刀磨利，

In [ ]:
new_cards[0]

In [51]:
r = fetch_code(code)

In [52]:
r

{'code': 0,
 'msg': '',
 'message': '',
 'data': {'new_num': 37,
  'exist_gap': 1,
  'update_num': 37,
  'open_rcmd': 1,
  'extra_flag': {'great_dynamic': 0},
  'cards': [{'desc': {'uid': 54992199,
     'type': 8,
     'rid': 93474806,
     'acl': 0,
     'view': 286757,
     'repost': 45,
     'like': 9295,
     'is_liked': 0,
     'dynamic_id': 362543940421790164,
     'timestamp': 1583249739,
     'pre_dy_id': 0,
     'orig_dy_id': 0,
     'orig_type': 0,
     'user_profile': {'info': {'uid': 54992199,
       'uname': '观视频工作室',
       'face': 'https://i0.hdslb.com/bfs/face/9fbaccf114c8917dcdebc118fb7056d95e8c0932.jpg'},
      'card': {'official_verify': {'type': 0, 'desc': 'bilibili 知名UP主'}},
      'vip': {'vipType': 2,
       'vipDueDate': 1609689600000,
       'dueRemark': '',
       'accessStatus': 0,
       'vipStatus': 1,
       'vipStatusWarn': '',
       'themeType': 0,
       'label': {'path': ''}},
      'pendant': {'pid': 0, 'name': '', 'image': '', 'expire': 0},
      'ra